In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from datasets import load_dataset

In [2]:
# Load civil_comments dataset
dataset = load_dataset("google/civil_comments")
print(dataset['train'][0])

{'text': "This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!", 'toxicity': 0.0, 'severe_toxicity': 0.0, 'obscene': 0.0, 'threat': 0.0, 'insult': 0.0, 'identity_attack': 0.0, 'sexual_explicit': 0.0}


In [3]:
# load data
df = pd.DataFrame(dataset['train'])

# multi-label
labels = df[df.columns[2:]].values

# texts
texts = df['text'].astype(str).valuess


In [4]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text

# Apply cleaning
df['text'] = df['text'].map(lambda x: clean_text(x))

In [5]:
df['text'].isnull().sum()

np.int64(0)

In [6]:
df = df[df['text'].str.strip() != '']

In [7]:
len(texts)

1804874

In [ ]:
# Create a vectorizer
vectorizer = TextVectorization(
    max_tokens=20000,
    output_sequence_length=300,  # Reduce length to reduce memory consumption
    output_mode='int'
)

# Training the vectorizer on texts
vectorizer.adapt(texts)

In [70]:
from sklearn.model_selection import train_test_split

texts_train, texts_temp, labels_train, labels_temp = train_test_split(
    texts, labels, test_size=0.3, random_state=42)

texts_val, texts_test, labels_val, labels_test = train_test_split(
    texts_temp, labels_temp, test_size=1/3, random_state=42)

# Create the datasets:
train_ds = tf.data.Dataset.from_tensor_slices((texts_train, labels_train))
val_ds = tf.data.Dataset.from_tensor_slices((texts_val, labels_val))
test_ds = tf.data.Dataset.from_tensor_slices((texts_test, labels_test))

AUTOTUNE = tf.data.AUTOTUNE

def prepare(ds):
    return ds.map(lambda x, y: (vectorizer(x), y), num_parallel_calls=AUTOTUNE)\
             .cache().shuffle(1805874).batch(32).prefetch(AUTOTUNE)

train = prepare(train_ds)
val = prepare(val_ds)
test = prepare(test_ds)


In [9]:
# Convert data to Dataset
dataset = tf.data.Dataset.from_tensor_slices((texts, labels))

dataset = dataset.map(lambda x, y: (vectorizer(x), y), num_parallel_calls=tf.data.AUTOTUNE)

# Data organization
dataset = dataset.cache()
dataset = dataset.shuffle(1805000)
dataset = dataset.batch(32)
dataset = dataset.prefetch(tf.data.AUTOTUNE)

In [13]:
for x_batch, y_batch in dataset.take(1):
    print("x_batch shape:", x_batch.shape)
    print("y_batch shape:", y_batch.shape)



x_batch shape: (32, 300)
y_batch shape: (32, 6)


In [14]:
len(dataset)

56403

In [16]:
batch_X , batch_Y = dataset.as_numpy_iterator().next()

In [22]:
batch_Y.shape

(32, 6)

In [19]:
batch_X.shape

(32, 300)

In [21]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dropout, Bidirectional, Dense, Embedding

In [24]:
model = Sequential()
model.add(Input(shape=(300,))) 
# Create the embedding layer 
model.add(Embedding(20000+1, 32))
# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation='tanh')))
# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# Final layer 
model.add(Dense(6, activation='sigmoid'))

In [25]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')


In [26]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 300, 32)             │         640,032 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 64)                  │          16,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 731,686 (2.79 MB)

 Trainable params: 731,686 (2.79 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
history = model.fit(train, epochs=5, validation_data=val)

Epoch 1/5
  582/39482 ━━━━━━━━━━━━━━━━━━━━ 1:21:43 126ms/step - loss: 0.1548

InvalidArgumentError: Graph execution error:

Detected at node IteratorGetNext defined at (most recent call last):
  File "C:\Users\tamer\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main

  File "C:\Users\tamer\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code

  File "C:\Users\tamer\OneDrive - Høgskulen på Vestlandet\Dokumenter\3 År\DAT255 Deep learning\toxicity2\toxicityPython39\lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\tamer\OneDrive - Høgskulen på Vestlandet\Dokumenter\3 År\DAT255 Deep learning\toxicity2\toxicityPython39\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\tamer\OneDrive - Høgskulen på Vestlandet\Dokumenter\3 År\DAT255 Deep learning\toxicity2\toxicityPython39\lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\tamer\OneDrive - Høgskulen på Vestlandet\Dokumenter\3 År\DAT255 Deep learning\tox

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(8,5))
pd.DataFrame(history.history).plot()
plt.show()